<a href="https://colab.research.google.com/github/GMouraBFR/Analise_RealMadrid_Barcelona/blob/main/Analise_RealMadrid_Barcelona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANÁLISE DOS BALANÇOES FINANCEIROS DE REAL MADRID E BARCELONA

In [3]:
# Instalar as dependências
!pip install plotly pandas numpy

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


Criando DataSet dos Clubes

In [5]:
data = {
    'Metric': ['Receita Total (M€)', 'EBITDA (M€)', 'Dívida Líquida (M€)', 'Wage-to-Revenue (%)',
               'Liquidez Corrente', 'Patrimônio Líquido (M€)', 'Valor de Mercado do Elenco (M€)'],
    'Real Madrid': [843, 180, 265, 54, 1.8, 534, 1030],
    'Barcelona': [800, 100, 680, 65, 0.7, 200, 860]
}

df = pd.DataFrame(data)

Análise Comparativa Visual

In [6]:
# Gráfico de Barras Comparativas
fig = px.bar(df,
             x='Metric',
             y=['Real Madrid', 'Barcelona'],
             barmode='group',
             title='Comparação Financeira: Real Madrid vs Barcelona',
             labels={'value': 'Valor', 'variable': 'Clube'},
             height=600)

fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Análise de Risco Financeiro

In [7]:
# Criar subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Dívida Líquida/EBITDA', 'Wage-to-Revenue Ratio'))

# Dados calculados
risk_data = {
    'Metric': ['Dívida/EBITDA', 'Wage-to-Revenue'],
    'Real Madrid': [265/180, 54],
    'Barcelona': [680/100, 65]
}

# Adicionar gráficos
fig.add_trace(go.Bar(
    x=risk_data['Metric'],
    y=risk_data['Real Madrid'],
    name='Real Madrid',
    marker_color='#004D98'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=risk_data['Metric'],
    y=risk_data['Barcelona'],
    name='Barcelona',
    marker_color='#A50044'
), row=1, col=1)

# Linhas de referência
fig.add_hline(y=4, line_dash="dot", line_color="red", annotation_text="Limite de Risco Alto", row=1, col=1)
fig.add_hline(y=70, line_dash="dot", line_color="red", annotation_text="Limite La Liga", row=1, col=2)

fig.update_layout(height=500, width=1000, showlegend=True)
fig.show()

Projeção de Receitas (2023-2026)

In [8]:
# Dados de projeção
years = [2023, 2024, 2025, 2026]
rm_projection = [843, 920, 1050, 1150]
fcb_projection = [800, 850, 900, 950]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=years,
    y=rm_projection,
    name='Real Madrid',
    line=dict(color='#004D98', width=4),
    mode='lines+markers'
))

fig.add_trace(go.Scatter(
    x=years,
    y=fcb_projection,
    name='Barcelona',
    line=dict(color='#A50044', width=4),
    mode='lines+markers'
))

fig.update_layout(
    title='Projeção de Receitas 2023-2026 (M€)',
    xaxis_title='Ano',
    yaxis_title='Receita Total (M€)',
    template='plotly_white'
)

fig.show()

Análise de Liquidez (Radar Chart)

In [9]:
metrics = ['Receita', 'EBITDA', 'Liquidez', 'Patrimônio', 'Valor de Mercado']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=[df[df['Metric'] == 'Receita Total (M€)']['Real Madrid'].values[0]/1000,
       df[df['Metric'] == 'EBITDA (M€)']['Real Madrid'].values[0]/100,
       df[df['Metric'] == 'Liquidez Corrente']['Real Madrid'].values[0],
       df[df['Metric'] == 'Patrimônio Líquido (M€)']['Real Madrid'].values[0]/1000,
       df[df['Metric'] == 'Valor de Mercado do Elenco (M€)']['Real Madrid'].values[0]/1000],
    theta=metrics,
    fill='toself',
    name='Real Madrid'
))

fig.add_trace(go.Scatterpolar(
    r=[df[df['Metric'] == 'Receita Total (M€)']['Barcelona'].values[0]/1000,
       df[df['Metric'] == 'EBITDA (M€)']['Barcelona'].values[0]/100,
       df[df['Metric'] == 'Liquidez Corrente']['Barcelona'].values[0],
       df[df['Metric'] == 'Patrimônio Líquido (M€)']['Barcelona'].values[0]/1000,
       df[df['Metric'] == 'Valor de Mercado do Elenco (M€)']['Barcelona'].values[0]/1000],
    theta=metrics,
    fill='toself',
    name='Barcelona'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True
        )),
    showlegend=True,
    title='Análise Comparativa em Radar'
)

fig.show()

Vamos incluir em nossa análise APIs, Machine Learning, análise de sensibilidade e web scraping.

In [21]:
!pip install plotly pandas numpy requests scikit-learn beautifulsoup4 yfinance --quiet
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from bs4 import BeautifulSoup
from sklearn.linear_model import LogisticRegression
import yfinance as yf



Dataset Básico

In [22]:
data = {
    'Metric': ['Receita Total (M€)', 'EBITDA (M€)', 'Dívida Líquida (M€)', 'Wage-to-Revenue (%)', 'Liquidez Corrente', 'Patrimônio Líquido (M€)', 'Valor de Mercado do Elenco (M€)'],
    'Real Madrid': [843, 180, 265, 54, 1.8, 534, 1030],
    'Barcelona': [800, 100, 680, 65, 0.7, 200, 860]
}
df = pd.DataFrame(data)


Conexão com APIs Financeiras (Yahoo Finance + SportMonks)

In [23]:
def get_financial_data(api_choice):
    if api_choice == 'Yahoo Finance':
        # Dados macroeconômicos (ex.: taxa de câmbio EUR/USD)
        eur_usd = yf.Ticker("EURUSD=X").history(period="1y")['Close'].iloc[-1]
        print(f"Última taxa EUR/USD: {eur_usd:.2f}")

    elif api_choice == 'SportMonks (Exemplo)':
        # Substituir 'YOUR_API_KEY' por uma chave válida do SportMonks
        url = "https://api.sportmonks.com/v3/football/clubs/82"
        headers = {"Authorization": "Bearer YOUR_API_KEY"}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            print(f"Dados do Real Madrid via API: {data['data']['name']}")
        else:
            print("Erro na API SportMonks")


Machine Learning: Previsão de Falência (Altman Z-Score)

In [24]:
data_altman = {
    'Clube': ['Real Madrid', 'Barcelona', 'Manchester United', 'Juventus'],
    'WC_TA': [0.15, -0.05, 0.10, 0.08],  # Capital de Giro / Ativos Totais
    'RE_TA': [0.30, -0.20, 0.25, 0.15],   # Lucros Retidos / Ativos Totais
    'EBIT_TA': [0.12, 0.05, 0.08, 0.06],  # EBIT / Ativos Totais
    'MVE_TL': [2.5, 0.8, 1.2, 1.0],       # Valor de Mercado / Passivo Total
    'Falido': [0, 1, 0, 0]                # 1 = Falência, 0 = Saudável
}
df_altman = pd.DataFrame(data_altman)
X = df_altman[['WC_TA', 'RE_TA', 'EBIT_TA', 'MVE_TL']]
y = df_altman['Falido']


Treinamento do modelo de regressão logística

In [25]:
model = LogisticRegression()
model.fit(X, y)


LogisticRegression()

In [26]:
# Previsão para Barcelona 2024
novo_clube_barcelona = np.array([[0.10, -0.15, 0.07, 0.9]])  # Exemplo: Barcelona 2024
novo_clube_barcelona_df = pd.DataFrame(novo_clube_barcelona, columns=['WC_TA', 'RE_TA', 'EBIT_TA', 'MVE_TL'])
predicao_barcelona = model.predict_proba(novo_clube_barcelona_df)[0][1]
print(f"\nProbabilidade de falência (Barcelona 2024): {predicao_barcelona*100:.1f}%")

# Previsão para Real Madrid 2024
novo_clube_real_madrid = np.array([[0.15, 0.30, 0.12, 2.5]]) # Exemplo: Real Madrid 2024
novo_clube_real_madrid_df = pd.DataFrame(novo_clube_real_madrid, columns=['WC_TA', 'RE_TA', 'EBIT_TA', 'MVE_TL'])
predicao_real_madrid = model.predict_proba(novo_clube_real_madrid_df)[0][1]
print(f"\nProbabilidade de falência (Real Madrid 2024): {predicao_real_madrid*100:.1f}%")



Probabilidade de falência (Barcelona 2024): 30.0%

Probabilidade de falência (Real Madrid 2024): 15.9%


Análise de Sensibilidade das Dívidas

In [32]:
def debt_sensitivity_analysis(club_name, debt_variations):
    base_data = df.set_index('Metric')[club_name]

    results = []
    for variation in debt_variations:
        modified_data = base_data.copy()
        modified_data['Dívida Líquida (M€)'] += variation
        modified_data['Endividamento'] = modified_data['Dívida Líquida (M€)'] / modified_data['Patrimônio Líquido (M€)']
        modified_data['Lucro Líquido'] = modified_data['EBITDA (M€)'] - modified_data['Dívida Líquida (M€)'] * 0.05  # Supondo 5% de juros
        results.append(modified_data)

    results_df = pd.DataFrame(results, index=[f'{variation:+}M€' for variation in debt_variations])
    return results_df


Realização da Análise de Sensibilidade para Real Madrid e Barcelona

In [30]:
# Variações na dívida em milhões de euros
debt_variations = [-100, -50, 0, +50, +100]

# Análise para Real Madrid
real_madrid_sensitivity = debt_sensitivity_analysis('Real Madrid', debt_variations)
print("Análise de Sensibilidade das Dívidas - Real Madrid:")
print(real_madrid_sensitivity)

# Análise para Barcelona
barcelona_sensitivity = debt_sensitivity_analysis('Barcelona', debt_variations)
print("\nAnálise de Sensibilidade das Dívidas - Barcelona:")
print(barcelona_sensitivity)


Análise de Sensibilidade das Dívidas - Real Madrid:
Metric  Receita Total (M€)  EBITDA (M€)  Dívida Líquida (M€)  \
-100M€               843.0        180.0                165.0   
-50M€                843.0        180.0                215.0   
+0M€                 843.0        180.0                265.0   
+50M€                843.0        180.0                315.0   
+100M€               843.0        180.0                365.0   

Metric  Wage-to-Revenue (%)  Liquidez Corrente  Patrimônio Líquido (M€)  \
-100M€                 54.0                1.8                    534.0   
-50M€                  54.0                1.8                    534.0   
+0M€                   54.0                1.8                    534.0   
+50M€                  54.0                1.8                    534.0   
+100M€                 54.0                1.8                    534.0   

Metric  Valor de Mercado do Elenco (M€)  Endividamento  Lucro Líquido  
-100M€                           1030.0 

Visualização dos Resultados

In [31]:
# Visualizar Análise de Sensibilidade das Dívidas para Real Madrid
fig_rm = px.line(real_madrid_sensitivity.T, title='Análise de Sensibilidade das Dívidas - Real Madrid')
fig_rm.update_layout(xaxis_title='Métrica', yaxis_title='Valor')
fig_rm.show()

# Visualizar Análise de Sensibilidade das Dívidas para Barcelona
fig_ba = px.line(barcelona_sensitivity.T, title='Análise de Sensibilidade das Dívidas - Barcelona')
fig_ba.update_layout(xaxis_title='Métrica', yaxis_title='Valor')
fig_ba.show()
